# ETL_EDA    
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from LAPD     
The [dataset](https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4) is a continuously updated file in csv format. It contains City of Los Angeles arrest data from 2010 to present. It can be retrieved simply by making requests without parameters. For more complicated API call, you may need to add query parameters.

In [4]:
import logging
import pandas as pd
from sodapy import Socrata
import utils

# download data
CRIME_SOURCE = "data.lacity.org"
client0 = Socrata(CRIME_SOURCE, None)
results = client0.get("yru6-6re4",where="arst_date > '2018-12-01T00:00:00.000'",limit = 1000000) # wait to be confirmed
print(results[:2])
print('...')
print(results[-2:])

[{'rpt_id': '191919936', 'arst_date': '2019-11-02T00:00:00.000', 'time': '1605', 'area': '19', 'area_desc': 'Mission', 'rd': '1972', 'age': '62', 'sex_cd': 'M', 'descent_cd': 'W', 'chrg_grp_cd': '06', 'grp_description': 'Larceny', 'arst_typ_cd': 'M', 'charge': '490.1PC', 'chrg_desc': 'PETTY THEFT UNDER $50.00', 'location': '9000    SEPULVEDA                    BL', 'location_1': {'latitude': '34.2337', 'longitude': '-118.4676', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}}, {'rpt_id': '5789339', 'arst_date': '2019-11-02T00:00:00.000', 'time': '2300', 'area': '18', 'area_desc': 'Southeast', 'rd': '1842', 'age': '18', 'sex_cd': 'M', 'descent_cd': 'B', 'chrg_grp_cd': '12', 'grp_description': 'Weapon (carry/poss)', 'arst_typ_cd': 'F', 'charge': '25850(A)PC', 'chrg_desc': 'CARRY LOADED F/ARM PRSON/HEH PUBLIC PLACE', 'location': '400 W  111TH                        ST', 'location_1': {'latitude': '33.9348', 'longitude': '-118.2811', 'human_address': '{"address": ""

## 2. Raw Data to Documents/Dicts      
Datetime conversion is made and age column is changed to numeric data type. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [5]:
from datetime import datetime

df = pd.DataFrame.from_records(results)
df['arst_date'] = pd.to_datetime(df['arst_date'])
df['hour'] = df['time'].apply(lambda x: datetime.strptime(x, '%H%M').time().hour)
df['age'] = pd.to_numeric(df['age'])

df.head()

,rpt_id,arst_date,time,area,area_desc,rd,age,sex_cd,descent_cd,chrg_grp_cd,grp_description,arst_typ_cd,charge,chrg_desc,location,location_1,crsst,hour
0,191919936,2019-11-02,1605,19,Mission,1972,62,M,W,06,Larceny,M,490.1PC,PETTY THEFT UNDER $50.00,9000 SEPULVEDA BL,"{'latitude': '34.2337', 'longitude': '-118.467...",NaN,16
1,5789339,2019-11-02,2300,18,Southeast,1842,18,M,B,12,Weapon (carry/poss),F,25850(A)PC,CARRY LOADED F/ARM PRSON/HEH PUBLIC PLACE,400 W 111TH ST,"{'latitude': '33.9348', 'longitude': '-118.281...",NaN,23
2,5789301,2019-11-02,2125,18,Southeast,1823,28,M,H,24,Miscellaneous Other Violations,F,236PC,FALSE IMPRISONMENT,400 E 93RD ST,"{'latitude': '33.952', 'longitude': '-118.2673...",NaN,21
3,5789240,2019-11-02,1400,03,Southwest,0357,52,M,B,08,Other Assaults,F,243(C)(2)PC,BATTERY ON PEACE OFFICER,1100 EXPOSITION BL,"{'latitude': '34.0219', 'longitude': '-118.295...",NaN,14
4,5789096,2019-11-02,1500,21,Topanga,2126,19,M,W,08,Other Assaults,F,422(A)PC,TERRORIZE CAUSING FEAR,7800 DE SOTO AV,"{'latitude': '34.212', 'longitude': '-118.5885...",NaN,15


## 3. Upsert MongoDB         
To avoid duplicate data entries between each run, the de-duplication mechanism makes sure that only a new record with a different report_id (rpt_id) will be inserted to the database. 

In [6]:
import pymongo
client = pymongo.MongoClient()
db = client.get_database("crime")
collection = db.get_collection("crime")
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter = {'rpt_id': record['rpt_id']},    # locate the document if exists
        replacement = record,                         # latest document
        upsert=True)
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

rows=7349, update=7349, insert=0
